In [2]:
#Install various packages if they are not already available



#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install beautifulsoup4
!pip install lxml
!pip install requests

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                       

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

import folium # map rendering library

from bs4 import BeautifulSoup


### The problem statement is :
## Make a comparative study between two cities of USA to make a decision on where to open a restaurent and with what kind of theme

### Choose two cities with similar population base
In our case we have chosen two mid-west cities of USA namely Chicago and Houston. Ther population base is on similar ranges as per Wikipedia

In [4]:
# Wikipedia URL below which will be the source of information of the two cities
CityComparisonURL ="https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population" 

In [5]:
#Parse the HTML from Wikipedia site and extract the population figures 
source = requests.get(CityComparisonURL).text
soup = BeautifulSoup (source, 'lxml')
CityColumns = ["CityName", "CityState", "CityPopulation", "CityLatitude", "CityLongitude"]
# instantiate the dataframe
ComparingCities = pd.DataFrame(columns=CityColumns)

#match = soup.title.text
USCitiesWikiTable = soup.find_all('tbody')
USCitiesWikiTable= USCitiesWikiTable[4]
#print(USCitiesWikiTable.prettify())

USCityRows = USCitiesWikiTable.find_all('tr')
#print(USCityRows[1].prettify())

for usCity in USCityRows[3:5] :
    #print (usCity.prettify())
    CityDetails = usCity.find_all('td')
    
    CityName = CityDetails[1].find('a')['title']
    CityState = CityDetails[2].find('a')['title']
    CityPopulation = CityDetails[3].text.replace(',','')[:-1]
    CityCoordinates= CityDetails[-1].find('span', class_="geo").text
    CityLatitude = CityCoordinates.split(";")[0]
    CityLongitude = CityCoordinates.split(";")[1]
    #print(CityName)
    #print(CityLatitude)
    ComparingCities = ComparingCities.append({'CityName': CityName, 'CityState': CityState, 
                                              'CityPopulation': CityPopulation, 'CityLatitude': CityLatitude , 'CityLongitude' : CityLongitude}, ignore_index=True)

ComparingCities

,CityName,CityState,CityPopulation,CityLatitude,CityLongitude
0,Chicago,Illinois,2705994,41.8376,-87.6818
1,Houston,Texas,2325502,29.7866,-95.3909


### As seen above , the population base of the two cities are comparable.

In [6]:
#Get Latitude & Longitudes for the two cities
address = 'Chicago, IL'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

address = 'Houston, TX'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Chicago, IL are 41.8755616, -87.6244212.
The geograpical coordinate of Houston, TX are 29.7589382, -95.3676974.


### The above two operations validates that the Coordinates of the two cities obtained from Wikipedia and Nominatim are identical

#### The next cell is marked with @hidden_cell. This cell contains my credentials of Foursquare. Since I do not want that to be revealed, I have marked this cell with @hidden_cell. This will redact the cell when it is pushed to Github

In [7]:
# The code was removed by Watson Studio for sharing.

## Explore the neighbourhood of the two cities

In [10]:
#First create a routine to get the list of facilities
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#        print(name)
            
        # create the API request URL
#        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#            CLIENT_ID, 
#            CLIENT_SECRET, 
#            VERSION, 
#            lat, 
#            lng, 
#            radius, 
#            LIMIT)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&radius={}&section={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            name, 
            radius, "food", 
            LIMIT)
        
        #print (url)   
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
City_venues = getNearbyVenues(names=ComparingCities['CityName'], latitudes=ComparingCities['CityLatitude'],
                                   longitudes=ComparingCities['CityLongitude'])

In [51]:
City_venues.size

588

In [13]:
 City_venues.groupby(["Neighborhood", "Venue Category"]).count()

Neighborhood Latitude  \
Neighborhood Venue Category                                           
Chicago      American Restaurant                                  2   
             Bakery                                               2   
             Burger Joint                                         1   
             Café                                                 1   
             Cantonese Restaurant                                 1   
             Chinese Restaurant                                   2   
             Diner                                                1   
             Donut Shop                                           2   
             Fast Food Restaurant                                 1   
             Food Truck                                           3   
             Fried Chicken Joint                                  1   
             Hot Dog Joint                                        1   
             Italian Restaurant                                   1   
             Latin American Restaurant                            1   
             Mexican Restaurant                                   6   
             Pizza Place                                          5   
             Sandwich Place                                       3   
             Sushi Restaurant                                     1   
             Szechuan Restaurant                                  1   
             Taco Place                                           2   
             Taiwanese Restaurant                                 1   
             Vietnamese Restaurant                                2   
Houston      BBQ Joint                                            1   
             Bistro                                               1   
             Burger Joint                                         6   
             Café                                                 1   
             Deli / Bodega                                        1   
             Dumpling Restaurant                                  1   
             Empanada Restaurant                                  1   
             Food Court                                           1   
             Gastropub                                            2   
             Greek Restaurant                                     1   
             Italian Restaurant                                   5   
             Japanese Restaurant                                  1   
             Mexican Restaurant                                   4   
             New American Restaurant                              1   
             Pizza Place                                          3   
             Sandwich Place                                       4   
             Seafood Restaurant                                   1   
             Southern / Soul Food Restaurant                      2   
             Steakhouse                                           2   
             Sushi Restaurant                                     1   
             Taco Place                                           2   
             Thai Restaurant                                      1   

                                              Neighborhood Longitude  Venue  \
Neighborhood Venue Category                                                   
Chicago      American Restaurant                                   2      2   
             Bakery                                                2      2   
             Burger Joint                                          1      1   
             Café                                                  1      1   
             Cantonese Restaurant                                  1      1   
             Chinese Restaurant                                    2      2   
             Diner                                                 1      1   
             Donut Shop                                            2

In [14]:
Chicago_venues = City_venues.loc[City_venues['Neighborhood']=="Chicago"]
#Drop the latitude & Longitude columns and create a new Dataframe
City_venuesNames =  City_venues.drop (["Neighborhood Latitude", "Neighborhood Longitude", "Venue Latitude", "Venue Longitude"], axis=1)
Chicago_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chicago,41.8376,-87.6818,Del Toro,41.853213,-87.646477,Mexican Restaurant
1,Chicago,41.8376,-87.6818,Andorka's Sandwich Shop,41.853758,-87.646842,Sandwich Place
2,Chicago,41.8376,-87.6818,Connie's Pizza,41.849214,-87.640757,Pizza Place
3,Chicago,41.8376,-87.6818,Haisous Vietnamese Kitchen,41.857910,-87.653331,Vietnamese Restaurant
4,Chicago,41.8376,-87.6818,Ca Phe Da,41.858012,-87.653601,Vietnamese Restaurant
5,Chicago,41.8376,-87.6818,Don Pedro Carnitas,41.857970,-87.653820,Taco Place
6,Chicago,41.8376,-87.6818,Steak 'n Egger,41.852655,-87.655974,Diner
7,Chicago,41.8376,-87.6818,5411 Empanadas Truck,41.845764,-87.656728,Food Truck
8,Chicago,41.8376,-87.6818,Kristoffer's Café & Bakery,41.858123,-87.646558,Bakery
9,Chicago,41.8376,-87.6818,Punky's Pizza & Pasta,41.845381,-87.641756,Pizza Place


###From the collection of the venues for both the cities, let us filter out some of the venue types. We want to restrict out find to Restaurant. This will make the comparative study a little simpler

In [62]:
City_Restaurants = City_venuesNames[~City_venuesNames["Venue Category"].str.contains('Joint|Bistro|Court|Truck|Shop|Place|pub|Deli|Diner|Bakery|Caf')]

In [63]:
pd.pivot_table(City_Restaurants, index= "Venue Category", columns = "Neighborhood", fill_value=0, aggfunc=lambda x: np.count_nonzero(x))

Venue        
Neighborhood                    Chicago Houston
Venue Category                                 
American Restaurant                   2       0
Cantonese Restaurant                  1       0
Chinese Restaurant                    2       0
Dumpling Restaurant                   0       1
Empanada Restaurant                   0       1
Fast Food Restaurant                  1       0
Greek Restaurant                      0       1
Italian Restaurant                    1       5
Japanese Restaurant                   0       1
Latin American Restaurant             1       0
Mexican Restaurant                    6       4
New American Restaurant               0       1
Seafood Restaurant                    0       1
Southern / Soul Food Restaurant       0       2
Steakhouse                            0       2
Sushi Restaurant                      1       1
Szechuan Restaurant                   1       0
Taiwanese Restaurant                  1       0
Thai Restaurant                       0       1
Vietnamese Restaurant                 2       0

### From the above tabulation, the observations are as below.
#### 1. It appears that Mexican Restaurants are quite numerous in both the locations(6 and 5). Opening a Mexican Restaurant in either city is going to face competition
#### 2. There are 5 Italian restaurants in Houston but only 1 in Chicago. There is a potential business opportunity
#### 3. Chicago does not seem to have a steakhouse at all. That should be a business opportunity here

### Overall, recommendation should be to go for a Italian Restaurant or a steakhouse in Chicago

In [28]:
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
import folium # map rendering library



ChicagoCoordinates = ComparingCities.loc[ComparingCities['CityName'] == 'Chicago', ['CityLatitude','CityLongitude']]
ChicagoCoordinates
#ax1 = folium.Map(location = [41.8376, -87.6818], zoom_start=11)
#ax2 = geodataframe.plot(ax=ax2, column='pred', legend=True)
#plt.show()

,CityLatitude,CityLongitude
0,41.8376,-87.6818


In [46]:
Chicago_venues = City_venues.loc[City_venues['Neighborhood']=="Chicago"]
ChicagoLat = float(ChicagoCoordinates['CityLatitude'][0])
ChicagoLong = float(ChicagoCoordinates['CityLongitude'][0])
ChicagoLat
#ChicagoLong

41.8376

In [64]:
# create map of Chicago using latitude and longitude values

map_Chicago = folium.Map(location=[ChicagoLat, ChicagoLong], zoom_start=13)

# add markers to map
for lat, lng, label in zip(Chicago_venues['Venue Latitude'], Chicago_venues['Venue Longitude'] , Chicago_venues['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7, parse_html=False).add_to(map_Chicago)  
    
map_Chicago

### Now to home in the choice of locality to open the restaurant
#### Visually, from the map of Chicago it appears that there are two chusters of location where the restaurants are present in large numbers
It will be the recommendation to chose between one of the clusters because, there must be a reason why the restaurants are operational there. 
There probability of foot-falls in the new restaurants will be more if it is located in one of the clusters. 
Opening the new restaurants in a far-off place will not help ! People of Chicago will typically throng to the area where there is a presence of existing restaurants
This will draw some proportion of the croud to the new Restaurant.

## End of Report